In [1]:




import shutil
from glob import glob
import os
from multiprocessing import Pool
from functools import partial

import numpy as np
from tqdm import tqdm
import pandas as pd
import re

In [80]:
path_data = r'E:/MIA/AMOS-CT-MR/valid/first_round'
path_to = r'E:/MIA/AMOS-CT-MR/'

In [81]:
path_data = glob(path_data + '/20*')

In [88]:
path_data

['E:/MIA/AMOS-CT-MR/valid/first_round\\2018',
 'E:/MIA/AMOS-CT-MR/valid/first_round\\2019',
 'E:/MIA/AMOS-CT-MR/valid/first_round\\2020']

In [ ]:
for path in path_data:
    os.makedirs(os.path.join(path_to, path.split(os.sep)[-1]))

In [ ]:
glob(path_data[0]+'\\*\\*')

In [55]:
data_list = []

for path in path_data:
    data_list.extend(glob(path+'\\*\\*'))

In [85]:
def move_(x):
     shutil.move(x, os.path.join(path_to, x.split(os.sep)[-3], x.split(os.sep)[-1]))

In [87]:
for x in data_list:
    shutil.move(x, os.path.join(path_to, x.split(os.sep)[-3], x.split(os.sep)[-1]))

In [96]:
df = pd.concat(pd.read_excel(r'E:\MIA\AMOS-CT-MR\valid\first_round\meta\firstround_data_meta-2019.xlsx', sheet_name=None, usecols=['Nii_File', '病历号']),ignore_index=True)

In [93]:
df.to_excel('2019_valid_segmentation_flags.xlsx', index=False)

In [95]:
df_c = pd.read_excel('../../data/2019_valid_segmentation_flags.xlsx')

In [98]:
df.merge(df_c, how='inner', on='Nii_File').to_excel('../../data/2019_valid_segmentation_flags.xlsx')

In [104]:
# dfs_2018 = pd.read_excel(r'..\..\data\firstround_data_meta-2018.xlsx', sheet_name=None)
dfs_2019 = pd.read_excel(r'..\..\data\firstround_data_meta-2019.xlsx', sheet_name=None)
dfs_2020 = pd.read_excel(r'..\..\data\firstround_data_meta-2020.xlsx', sheet_name=None)

In [25]:
r = re.compile("2018.")
cols = list(filter(r.match, dfs_2018.keys()))

In [27]:
cols

['201808', '201809', '201810', '201811', '201812']

In [75]:
def write(out_name, dfs):
    writer = pd.ExcelWriter(out_name, engine='xlsxwriter')
    for key in dfs.keys():
        dfs[key].to_excel(writer, sheet_name=key, index = False)
    writer.save()

In [ ]:
df_ref_2019 = pd.read_excel(r'../../data/2019_valid_segmentation_flags.xlsx')
df_ref_2020 = pd.read_excel(r'../../data/firstround_data2020-pre.xlsx')

In [78]:
#columns = {'flag':'complete_ab_flag'}, inplace = True
def cb_ann_half(df, df_ref):
    df.drop(df.columns[0], axis=1, inplace=True)
    if('flag' in df.columns):
        df.rename(columns = {'flag':'complete_ab_flag'}, inplace = True, errors="raise")
    if(df_ref.shape[0] == 0 and 'ann_flag' not in df.columns):
        df['ann_flag'] = ""
        return df
    df_ref = df_ref[['ann_flag', 'Nii_File']]
    df_out = pd.merge(df, df_ref, how='left', on='Nii_File')
    return df_out

In [71]:
def op_df(dfs, df_ref):
    for key in dfs:
        dfs[key] = cb_ann_half(dfs[key], df_ref)
    return dfs

In [ ]:
write('firstround_data_meta-2018.xlsx',op_df(dfs_2018, pd.DataFrame()))
write('firstround_data_meta-2019.xlsx',op_df(dfs_2019, df_ref_2019))
write('firstround_data_meta-2020.xlsx',op_df(dfs_2020, df_ref_2020))

In [105]:
def iter_cat_df(dfs):
    res = []
    for key in dfs:
        df = dfs[key]
        res.append(df[df['ann_flag']==1]['病历号'])
        # res.append(df['病历号'])
    return pd.concat(res, axis=0)


In [97]:
pids_2019 = iter_cat_df(dfs_2019)
pids_2020 = iter_cat_df(dfs_2020)

In [106]:
pids_ann_2019 = iter_cat_df(dfs_2019)
pids_ann_2020 = iter_cat_df(dfs_2020)

In [107]:
ann_counts = pd.concat([pids_ann_2019, pids_ann_2020]).value_counts()

In [98]:
counts = pd.concat([pids_2019, pids_2020]).value_counts()

In [108]:
ann_counts[ann_counts != 1]

Series([], Name: 病历号, dtype: int64)

In [100]:
'59a55a2d04db85d16d51aca4c82879db' in ann_counts

True

In [103]:
'8954981e037138a068726293fc78e2bf' in pids_ann_2020

False

In [109]:
len(ann_counts)

100